In [ ]:
import pandas as pd

In [ ]:
df_basics = pd.read_csv("/home/t-merty/mounts/sandbox-mert/data/ImdbTitleBasics.csv")

In [ ]:
df_crew = pd.read_csv("/home/t-merty/mounts/sandbox-mert/data/ImdbTitlePrincipals.csv")
df_crew = df_crew[df_crew.category.isin(["actor", "actress"])]

In [ ]:
df_names = pd.read_csv("/home/t-merty/mounts/sandbox-mert/data/ImdbName.csv")

In [ ]:
df_basics.columns, df_crew.columns

In [ ]:
df_crew = pd.merge(df_crew, df_basics[['tconst', 'primaryTitle']], on='tconst', how='left')
df_crew = pd.merge(df_crew, df_names[['nconst', 'primaryName']], on='nconst', how='left')


In [ ]:
df_basics[df_basics.titleType == "movie"]

In [ ]:
import json

In [ ]:
nobel_json = json.load(open("/home/t-merty/mounts/sandbox-mert/data/nobel_winners.json"))

In [ ]:
df_nobel = pd.DataFrame(nobel_json)

In [ ]:
df_nobel.head()

In [ ]:
df_nobel[df_nobel.borncity == "Todmorden"]

In [ ]:
import pickle
nobel_items = []
for city in df_nobel.borncity.unique():
    prompt = f"User: Is there a person who was born in the city of {city} and who is a Nobel Prize Winner\nAssistant: Yes, the person's name is"
    nobel_items.append({"constraints": [" who is a Nobel Prize Winner", f" who was born in the city of {city}"],
                        "prompt": prompt, 
                        "popularity": -1
                       })
    print(prompt)

with open("/home/t-merty/mounts/sandbox-mert/data/nobel_data.pkl", "wb") as f:
    pickle.dump(nobel_items, f)


In [ ]:
with open("/home/t-merty/mounts/sandbox-mert/data/nobel_data.pkl", "rb") as f:
    nobel_items = pickle.load(f)

In [ ]:
nobel_items

In [ ]:
import requests

query = """
SELECT DISTINCT ?senator ?senatorLabel ?stateLabel ?schoolLabel ?popularity WHERE {
  ?senator p:P39 ?statement.
  ?statement ps:P39 wd:Q4416090.
  OPTIONAL { ?statement pq:P768 ?state. }  # <- state represented
  ?senator wdt:P31 wd:Q5;  # <- instance of human
           p:P69 ?educationStatement;  # <- educational institution
           wikibase:sitelinks ?popularity.   # <- popularity (site links)
  ?educationStatement ps:P69 ?school.  # <- school
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}

"""

url = "https://query.wikidata.org/sparql"
params = {"query": query, "format": "json"}

response = requests.get(url, params=params)
data = response.json()


In [ ]:
all_senators = []
for i, item in enumerate(data['results']['bindings']):
    all_senators.append(item['senatorLabel']['value'])
all_senators = list(set(all_senators))
all_senators = [f"{i+1}. {s}" for i, s in enumerate(all_senators)]
all_senators_txt = "\n".join(all_senators)
records = []
for item in data['results']['bindings']:
    if "stateLabel" not in item:
        continue
    records.append({"Name": item['senatorLabel']['value'], 
                   "School": item['schoolLabel']['value'], 
                   "State": item['stateLabel']['value'].split(" Class")[0], 
                    "Popularity": item['popularity']['value'], 
                   })
df_all_senators = pd.DataFrame(records)


    

In [ ]:
hall_senators = pd.read_csv("./rare_knowledge/data/hallucination_senator.csv")
hall_senators["state"] = hall_senators["prompt"].apply(lambda x: x.split("state of ")[1].split(" ")[0])
hall_senators["school"] = hall_senators["prompt"].apply(lambda x: x.split("alma mater was ")[-1].split("?")[0])

In [ ]:
records = []
for item in data['results']['bindings']:
    if "stateLabel" not in item:
        continue
    if ("university" not in item['schoolLabel']['value'].lower()) and ("college" not in item['schoolLabel']['value'].lower()):
        continue
    if not (len(item["schoolLabel"]["value"].split(" ")) <= 2):
        continue
    records.append({"Name": item['senatorLabel']['value'], 
                   "School": item['schoolLabel']['value'], 
                   "State": item['stateLabel']['value'].split(" Class")[0], 
                    "Popularity": item['popularity']['value'], 
                   })
df_questions = pd.DataFrame(records)

In [ ]:
import numpy as np
num_to_sample = 5
items = []
df_questions.sort_values("Popularity", ascending=False, inplace=True)
for state in df_questions.State.unique():
    prompt_template = """Fill in the blanks.\nOccupation: Senator\nState: {}\nAlma Mater: {}\nName:"""
    hall_school = hall_senators[hall_senators.state == state].school.unique()    
    item_per_state = np.min([num_to_sample, len(hall_school), df_questions[df_questions.State == state].shape[0]])
    if len(hall_school) == 0:
        continue
    hall_schools = np.random.choice(hall_school, item_per_state, replace=False)
    for hall_school in hall_schools:
        hall_prompt = prompt_template.format(state, hall_school)
        items.append({"constraints": ["State: {}".format(state), "Alma Mater: {}".format(hall_school)], "prompt": hall_prompt, "exists": 0,
                     "popularity": -1})
    for i in range(item_per_state):
        exists_school = df_questions[df_questions.State == state].iloc[i]["School"]
        exists_prompt = prompt_template.format(state, exists_school)
        items.append({"constraints": ["State: {}".format(state), "Alma Mater: {}".format(exists_school)], "prompt": exists_prompt, "exists": 1,
                     "popularity": -1})


In [ ]:
with open("/home/t-merty/mounts/sandbox-mert/data/senator_multiconstraint_data.pkl", "wb") as f:
    pickle.dump(items, f)

In [ ]:
with open("/home/t-merty/mounts/sandbox-mert/data/senator_multiconstraint_data.pkl", "rb") as f:
    pickle.load(f)

## Words

In [ ]:
from tqdm import tqdm
from nltk.corpus import words
import random
import string

all_words = words.words()
def satisfies_conditions(word, conditions):
    for pos, letter in conditions:
        try:
            if word[pos] != letter: return False
        except IndexError:
            return False
    return True

def count_satisfying_words(conditions):
    return sum(satisfies_conditions(w, conditions) for w in all_words)


def sample_words():
    items = []
    constraint_set = list()
    all_letters = list(string.ascii_lowercase)
    
    for first in tqdm(all_letters):
        for last in all_letters:
            constraints = [f" starts with the letter {first}", f" ends with the letter {last}"]
            letters = [first, last]
        
            popularity = count_satisfying_words([[0, first], [-1, last]])
                
            full_prompt = f"User: Is there a word that{constraints[0]} and{constraints[1]}\nAssistant: Yes, one such word is"
            items.append({"prompt": full_prompt, "constraints": constraints,
                          "conditions": [[0, first], [-1, last]],
                         "popularity": popularity})

            constraints = [f" ends with the letter {last}", f" starts with the letter {first}"]
            letters = [first, last]
                        
            full_prompt = f"User: Is there a word that{constraints[0]} and{constraints[1]}\nAssistant: Yes, one such word is"
            items.append({"prompt": full_prompt, "constraints": constraints,
                          "conditions": [[0, first], [-1, last]],
                         "popularity": popularity})
            
    return items

In [ ]:
items = sample_words()

In [ ]:
with open("/home/t-merty/mounts/sandbox-mert/data/word_startend_multiconstraint_data.pkl", "wb") as f:
    pickle.dump(items, f)

In [ ]:
with open("/home/t-merty/mounts/sandbox-mert/data/word_startend_multiconstraint_data.pkl", "rb") as f:
    items = pickle.load(f)

In [ ]:
print(len(items))
